# I. Bayes Estimator and Bayes Risk

We simulate an experiment to determine the likelihood of a person being a professional football player, an amateur, or neither, based on the position they usually play. 

## Question 1 (M)

#### A. Setup

##### Input Space \(X\)
The input space \(X = \{0, 1, 2, 3\}\) represents the positions played on a football field:
- \(0\) - Goalkeeper
- \(1\) - Defender
- \(2\) - Midfielder
- \(3\) - Attacker

##### Output Space \(Y\)
The output space \(Y = \{0, 1, 2\}\) categorizes the football playing status of a person:
- \(0\) - Regular person
- \(1\) - Amateur
- \(2\) - Professional

##### Probability Model for \(X\)
\(X\) follows a multinomial distribution with a single trial and probabilities \([0.1, 0.2, 0.3, 0.4]\) for each of the positions respectively. This suggests that in our sample, attackers are most frequent, followed by midfielders, defenders, and goalkeepers.

#### B. Conditional Distributions \(Y | X\)
- $Y = Mult(1, [0.7, 0.2, 0.1])$ if $X = 0$
- $Y = Mult(1, [0.8, 0.15, 0.05])$ if $X = 1$
- $Y = Mult(1, [0.9, 0.08, 0.02])$ if $X >= 2$


#### C. Bayes Estimator


- $P(Y=0) = \sum_{i=0}^3 P(Y=0|X=x_i)*P(X=x_i) = 0.86$
- $P(Y=1) = \sum_{i=0}^3 P(Y=1|X=x_i)*P(X=x_i) = 0.106$
- $P(Y=2) = \sum_{i=0}^3 P(Y=2|X=x_i)*P(X=x_i) = 0.0304$

$f^*(x) = 0$

#### D. Bayes Risk

$R(f^*) = E[I(Y, f^*(x))] = \sum_{i=0}^3 P(Y \neq 0 | X=x_i)*P(X=x_i) = 0.14$

## Question 2 (C)

- Bayes estimator : $f^*(x) = 0$
- Estimator f1 : $f_1(x) = 2$ if $X=0$ and $0$ otherwise

In [ ]:
import numpy as np

def bayes() -> None:

    n_samples = int(1e6)

    X = np.random.multinomial(1, [0.1, 0.2, 0.3, 0.4], size=n_samples)

    # Copy X for parameters
    multinomial_parameters = np.zeros((n_samples, 3))  # Assuming there are 3 categories
    # Set parameters for each category using boolean indexing
    multinomial_parameters[X[:, 0] == 1] = [0.7, 0.2, 0.1]
    multinomial_parameters[X[:, 1] == 1] = [0.8, 0.15, 0.05]
    multinomial_parameters[(X[:, 2] == 1) | (X[:, 3] == 1)] = [0.9, 0.08, 0.02]

    # Generate new multinomial distribution based on updated parameters
    y = np.array([np.random.multinomial(1, p) for p in multinomial_parameters])

    y_pred_bayes = np.zeros((n_samples, 3))
    y_pred_bayes[:, 0] = 1

    # print(f"Bayes estimator: {y_pred_bayes}")
    y_pred_f1 = np.zeros((n_samples, 3))
    y_pred_f1[X[:, 0] == 1] = [0, 0, 1]
    y_pred_f1[(X[:, 1] == 1) | (X[:, 2] == 1) | (X[:, 3] == 1)] = [1, 0, 0]

    emperical_risk_bayes = np.mean(np.any(y != y_pred_bayes, axis=1))
    emperical_risk_f1 = np.mean(np.any(y != y_pred_f1, axis=1))

    print(f"Empirical risk for Bayes estimator: {emperical_risk_bayes}")
    print(f"Empirical risk for f1 estimator: {emperical_risk_f1}")

bayes()

# II. Bayes risk with absolute loss

## Question 1 (M)

- A function that respects the requirements is $f(x) = x^3$
- $f'(x) = 3x^2$ and $f'(x) = 0 \iff x = 0$